In [1]:
import pandas as pd
from pathlib import Path
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
import graphs

# Wczytanie danych
json_file = Path("../data/clean_data.json")
df = pd.read_json(json_file, orient='records', lines=True)

# Zachowanie kolumny `url` do późniejszego powiązania
urls = df["url"]

# Przygotowanie danych do treningu
df = df.drop(columns=["id", "lokalizacja", "url"], errors="ignore")
X = df.drop(columns=["cena"])  # Wszystkie kolumny poza 'cena' jako cechy
X = pd.get_dummies(X, columns=["model", "rodzaj_paliwa", "skrzynia_biegow", "naped", "typ_nadwozia", "kolor", "stan_pojazdu"], drop_first=True)
X[['rok_produkcji', 'przebieg', 'pojemnosc_silnika', 'moc_silnika', 'liczba_drzwi', 'liczba_miejsc']] = StandardScaler().fit_transform(X[['rok_produkcji', 'przebieg', 'pojemnosc_silnika', 'moc_silnika', 'liczba_drzwi', 'liczba_miejsc']])
y = df["cena"]  # Kolumna 'cena' jako cel

# Podział na zbiory treningowe i testowe
X_train, X_test, y_train, y_test, urls_train, urls_test = train_test_split(X, y, urls, test_size=0.2, random_state=42)


c:\studia\wejherowo_tree\wejherowo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

# 1. Trenowanie modelu XGBoost
model = XGBRegressor(
    n_estimators=100,  # Liczba drzew
    learning_rate=0.1,  # Szybkość uczenia
    max_depth=6,  # Maksymalna głębokość drzewa
    random_state=42
)
model.fit(X_train, y_train)

# 2. Predykcja na zbiorze treningowym i testowym
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 3. Obliczanie metryk
train_rmse = mean_squared_error(y_train, y_train_pred, squared=False)
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)

# 4. Obliczanie średniej procentowej rozbieżności
train_percentage_error = ((abs(y_train - y_train_pred) / y_train) * 100).mean()
test_percentage_error = ((abs(y_test - y_test_pred) / y_test) * 100).mean()

# 5. Wyświetlenie wyników
print("Wyniki modelu XGBoost:")
print(f"RMSE na zbiorze treningowym: {train_rmse:.2f}")
print(f"RMSE na zbiorze testowym: {test_rmse:.2f}")
print(f"Średnia procentowa rozbieżność na zbiorze treningowym: {train_percentage_error:.2f}%")
print(f"Średnia procentowa rozbieżność na zbiorze testowym: {test_percentage_error:.2f}%")


Wyniki modelu XGBoost:
RMSE na zbiorze treningowym: 16148.35
RMSE na zbiorze testowym: 23548.17
Średnia procentowa rozbieżność na zbiorze treningowym: 13.42%
Średnia procentowa rozbieżność na zbiorze testowym: 15.52%


c:\studia\wejherowo_tree\wejherowo\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\studia\wejherowo_tree\wejherowo\.venv\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
import optuna
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Funkcja celu dla Optuna
def objective(trial):
    # Propozycje hiperparametrów
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 0.0, 10.0),
        "reg_lambda": trial.suggest_float("reg_lambda", 0.0, 10.0),
    }
    
    # Tworzenie modelu XGBRegressor z zaproponowanymi parametrami
    model = XGBRegressor(random_state=42, **params)
    
    # Trenowanie modelu
    model.fit(X_train, y_train)
    
    # Predykcja na zbiorze testowym
    y_pred = model.predict(X_test)
    
    # Obliczanie metryki RMSE
    rmse = mean_absolute_error(y_test, y_pred)
    
    return rmse

# Tworzenie obiektu Optuna i optymalizacja
study = optuna.create_study(direction="minimize")  # Minimalizujemy RMSE
study.optimize(objective, n_trials=50)  # Liczba prób

# Wyświetlenie najlepszych parametrów i wyniku
print("Najlepsze parametry:")
print(study.best_params)
print(f"Najlepsze RMSE: {study.best_value:.2f}")

# Trenowanie finalnego modelu z najlepszymi parametrami
best_params = study.best_params
final_model = XGBRegressor(random_state=42, **best_params)
final_model.fit(X_train, y_train)

# Ocena modelu
y_test_pred = final_model.predict(X_test)

# Obliczenie metryk
test_rmse = mean_squared_error(y_test, y_test_pred, squared=False)
test_mae = mean_absolute_error(y_test, y_test_pred)  # Średni błąd w PLN
test_percentage_error = ((abs(y_test - y_test_pred) / y_test) * 100).mean()

# Wyświetlenie wyników
print(f"RMSE na zbiorze testowym: {test_rmse:.2f} PLN")
print(f"Średnia procentowa rozbieżność na zbiorze testowym: {test_percentage_error:.2f}%")
print(f"Średni błąd w PLN (MAE): {test_mae:.2f} PLN")
